In [1]:
# import libraries & read files
import pandas as pd
df_supply = pd.read_excel('INPUT_supply_planning.xlsx', sheet_name = 'Supply', skiprows = [0])
df_supply

,Part_No,Stock on hand,2022-03-28 00:00:00,2022-04-04 00:00:00,2022-04-11 00:00:00,2022-04-18 00:00:00,2022-04-25 00:00:00
0,BANH-XE,54,2388,4776.0,2393.0,4781.0,2398.0
1,CHUONG,0,408,816.0,413.0,821.0,418.0
2,BAN-DAP,0,2200,NaN,NaN,NaN,NaN
3,YEN-XE,41,1959,3918.0,1964.0,3923.0,1969.0


In [2]:
df_demand = pd.read_excel('INPUT_supply_planning.xlsx', sheet_name = 'Demand', usecols = 'A,C,D')
df_demand

,Part_No,Job Qty,Job start date
0,BANH-XE,2442,2022-04-11
1,BANH-XE,880,2022-05-03
2,CHUONG,408,2022-04-10
3,CHUONG,1292,2022-04-11
4,BAN-DAP,2200,2022-04-21
5,BAN-DAP,2500,2022-05-01
6,YEN-XE,2000,2022-04-13
7,YEN-XE,1542,2022-04-17
8,YEN-XE,248,2022-04-18


In [3]:
# check data info
df_supply.info()
df_demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Part_No              4 non-null      object 
 1   Stock on hand        4 non-null      int64  
 2   2022-03-28 00:00:00  4 non-null      int64  
 3   2022-04-04 00:00:00  3 non-null      float64
 4   2022-04-11 00:00:00  3 non-null      float64
 5   2022-04-18 00:00:00  3 non-null      float64
 6   2022-04-25 00:00:00  3 non-null      float64
dtypes: float64(4), int64(2), object(1)
memory usage: 352.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Part_No         9 non-null      object        
 1   Job Qty         9 non-null      int64         
 2   Job start date  9 non-null      datetime64[ns]
dtypes: datetime64[ns](

In [4]:
# check if df_demand has NaN value
df_demand.isna().sum()

Part_No           0
Job Qty           0
Job start date    0
dtype: int64

In [5]:
# rename df_demand columns
df_demand = df_demand.rename(columns = {'Job Qty':'demand_quantity','Job start date':'demand_date'})

In [6]:
# rename Stock on hand column to a date before 2022-03-28: 2022-03-21
df_supply = df_supply.rename(columns = {'Stock on hand':pd.to_datetime('2022-03-21')})

In [7]:
# unpivot df_supply
df_supply_unpivot = df_supply.melt('Part_No')
df_supply_unpivot.head(5)

,Part_No,variable,value
0,BANH-XE,2022-03-21,54.0
1,CHUONG,2022-03-21,0.0
2,BAN-DAP,2022-03-21,0.0
3,YEN-XE,2022-03-21,41.0
4,BANH-XE,2022-03-28,2388.0


In [8]:
# check if exist NaN value
df_supply_unpivot.isna().sum()

Part_No     0
variable    0
value       4
dtype: int64

In [9]:
# change NaN value to 0
df_supply_unpivot['value'] = df_supply_unpivot['value'].fillna(0)

In [10]:
# rename columns of df_supply
df_supply_unpivot = df_supply_unpivot.rename(columns = {'variable':'supply_date','value':'supply_quantity'})

In [11]:
# sort dataframes - ascending
df_demand = df_demand.sort_values(by = ['Part_No','demand_date'], ascending = True)
df_supply_unpivot = df_supply_unpivot.sort_values(by = ['Part_No','supply_date'], ascending = True)

In [12]:
# calculate cum_sum for each Part_No
df_demand['cum_sum_demand'] = df_demand.groupby('Part_No')['demand_quantity'].cumsum()
df_supply_unpivot['cum_sum_supply'] = df_supply_unpivot.groupby('Part_No')['supply_quantity'].cumsum()
df_supply_unpivot.head(5)

,Part_No,supply_date,supply_quantity,cum_sum_supply
2,BAN-DAP,2022-03-21,0.0,0.0
6,BAN-DAP,2022-03-28,2200.0,2200.0
10,BAN-DAP,2022-04-04,0.0,2200.0
14,BAN-DAP,2022-04-11,0.0,2200.0
18,BAN-DAP,2022-04-18,0.0,2200.0


In [14]:
# merge 2 dataframes by Part_No
df = pd.merge(df_demand, df_supply_unpivot, how = 'outer', on = 'Part_No')
df

,Part_No,demand_quantity,demand_date,cum_sum_demand,supply_date,supply_quantity,cum_sum_supply
0,BAN-DAP,2200,2022-04-21,2200,2022-03-21,0.0,0.0
1,BAN-DAP,2200,2022-04-21,2200,2022-03-28,2200.0,2200.0
2,BAN-DAP,2200,2022-04-21,2200,2022-04-04,0.0,2200.0
3,BAN-DAP,2200,2022-04-21,2200,2022-04-11,0.0,2200.0
4,BAN-DAP,2200,2022-04-21,2200,2022-04-18,0.0,2200.0
5,BAN-DAP,2200,2022-04-21,2200,2022-04-25,0.0,2200.0
6,BAN-DAP,2500,2022-05-01,4700,2022-03-21,0.0,0.0
7,BAN-DAP,2500,2022-05-01,4700,2022-03-28,2200.0,2200.0
8,BAN-DAP,2500,2022-05-01,4700,2022-04-04,0.0,2200.0
9,BAN-DAP,2500,2022-05-01,4700,2022-04-11,0.0,2200.0


In [18]:
# filter data where cum_sum_supply >= cum_sum_demand
df_filtered = df[df['cum_sum_supply']>=df['cum_sum_demand']]
df_filtered.head(10)

,Part_No,demand_quantity,demand_date,cum_sum_demand,supply_date,supply_quantity,cum_sum_supply
1,BAN-DAP,2200,2022-04-21,2200,2022-03-28,2200.0,2200.0
2,BAN-DAP,2200,2022-04-21,2200,2022-04-04,0.0,2200.0
3,BAN-DAP,2200,2022-04-21,2200,2022-04-11,0.0,2200.0
4,BAN-DAP,2200,2022-04-21,2200,2022-04-18,0.0,2200.0
5,BAN-DAP,2200,2022-04-21,2200,2022-04-25,0.0,2200.0
13,BANH-XE,2442,2022-04-11,2442,2022-03-28,2388.0,2442.0
14,BANH-XE,2442,2022-04-11,2442,2022-04-04,4776.0,7218.0
15,BANH-XE,2442,2022-04-11,2442,2022-04-11,2393.0,9611.0
16,BANH-XE,2442,2022-04-11,2442,2022-04-18,4781.0,14392.0
17,BANH-XE,2442,2022-04-11,2442,2022-04-25,2398.0,16790.0


In [22]:
# only takes the first row of each Part_No and cum_sum_demand
df_filtered = df_filtered.groupby(['Part_No','cum_sum_demand']).first().reset_index()
df_filtered

,Part_No,cum_sum_demand,demand_quantity,demand_date,supply_date,supply_quantity,cum_sum_supply
0,BAN-DAP,2200,2200,2022-04-21,2022-03-28,2200.0,2200.0
1,BANH-XE,2442,2442,2022-04-11,2022-03-28,2388.0,2442.0
2,BANH-XE,3322,880,2022-05-03,2022-04-04,4776.0,7218.0
3,CHUONG,408,408,2022-04-10,2022-03-28,408.0,408.0
4,CHUONG,1700,1292,2022-04-11,2022-04-18,821.0,2458.0
5,YEN-XE,2000,2000,2022-04-13,2022-03-28,1959.0,2000.0
6,YEN-XE,3542,1542,2022-04-17,2022-04-04,3918.0,5918.0
7,YEN-XE,3790,248,2022-04-18,2022-04-04,3918.0,5918.0


In [59]:
# filter when supply_date > demand_date
df_late = df_filtered[df_filtered['supply_date']>df_filtered['demand_date']]
df_late

,Part_No,cum_sum_demand,demand_quantity,demand_date,supply_date,supply_quantity,cum_sum_supply
4,CHUONG,1700,1292,2022-04-11,2022-04-18,821.0,2458.0


In [60]:
# calculate how many days delayed
df_late['delayed_days'] = df_delay['supply_date'] - df_delay['demand_date']
df_late

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15152\4207091651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_late['delayed_days'] = df_delay['supply_date'] - df_delay['demand_date']


,Part_No,cum_sum_demand,demand_quantity,demand_date,supply_date,supply_quantity,cum_sum_supply,delayed_days
4,CHUONG,1700,1292,2022-04-11,2022-04-18,821.0,2458.0,7 days


In [66]:
# quantity in short
df_late['quantity_in_short'] = df_late['demand_quantity'] - (df_late['cum_sum_supply']-df_late['supply_quantity'])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15152\2743717744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_late['quantity_in_short'] = df_late['demand_quantity'] - (df_late['cum_sum_supply']-df_late['supply_quantity'])


In [67]:
df_late

,Part_No,cum_sum_demand,demand_quantity,demand_date,supply_date,supply_quantity,cum_sum_supply,delayed_days,quantity_in_short
4,CHUONG,1700,1292,2022-04-11,2022-04-18,821.0,2458.0,7 days,-345.0
